In [45]:
#from rdfpandas.graph import to_dataframe
import shared.config as config
import pymongo
import numpy as np
import pandas as pd
import json
import re

#import rdflib
from ipywidgets import interact, Dropdown
from IPython.display import display
import ipywidgets as widgets

import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection

pd.set_option('display.max_rows', 100)
pd.set_option('max_colwidth', 400)

inputExcel = '/input/referentietabellen/abr_versie_01122018.xlsx'
outputExcel = '/input/referentietabellen/abr_versie_01122018_input.xlsx'
#ABRTurtle = './work/datasets/abr.ttl'

In [46]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
stagingDb = myclient[str(config.DB_STAGING)]
analyseDb = myclient[str(config.DB_ANALYSE)]
stagingCol = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
analyseCol = analyseDb[config.COLL_ANALYSE]
analyseColClean = analyseDb[config.COLL_ANALYSE_CLEAN]

metaCollection = stagingDb['Kolominformatie']

In [47]:
#g = rdflib.Graph()
#g.parse(ABRTurtle, format = 'ttl')
#df_import = to_dataframe(g)
#df.to_csv('test.csv', index = True, index_label = "@id")
#df_import

In [48]:
df_abr = pd.read_excel(inputExcel)
df_abr.reset_index(inplace=True)
df_abr.rename(columns={"index": "ID"}, inplace=True)
df_abr.dropna(subset=['uri'])
df_abr

,ID,uri,scheme,concept,concept.1,concept.2,concept.3,concept.4,concept.5,concept.6,...,https://data.cultureelerfgoed.nl/vocab/id/abr#hasBeginYear,https://data.cultureelerfgoed.nl/vocab/id/abr#hasEndYear,https://data.cultureelerfgoed.nl/vocab/id/abr#hasSpecificAbrCode,https://data.cultureelerfgoed.nl/vocab/id/abr#hasSpecificAbrCode@nl,https://data.cultureelerfgoed.nl/vocab/id/rce#hasCode,https://data.cultureelerfgoed.nl/vocab/id/rce#hasCode@nl,https://data.cultureelerfgoed.nl/vocab/id/rce#hasSourceId,https://data.cultureelerfgoed.nl/vocab/id/rce#hasSourceId@nl,https://data.cultureelerfgoed.nl/vocab/id/rce#hasSpecification,https://data.cultureelerfgoed.nl/vocab/id/rce#hasSpecification@nl
0,0,https://data.cultureelerfgoed.nl/term/id/abr/AlgemeneLijsten,Algemene Lijsten,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,https://data.cultureelerfgoed.nl/term/id/abr/bc241422-ee0d-4c44-aead-94a25b5e9fa4,NaN,bronhouder,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,https://data.cultureelerfgoed.nl/term/id/abr/17e56d62-b1f2-42e1-afbf-578d7b018380,NaN,NaN,Stichting Infrastructuur Kwaliteitsborging Bodembeheer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,https://data.cultureelerfgoed.nl/term/id/abr/1576adc6-1af1-44d0-9d46-593eb6bcad09,NaN,NaN,Rijksdienst voor het Cultureel Erfgoed,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,https://data.cultureelerfgoed.nl/term/id/abr/d159858d-d746-4de5-98be-82cc5b697f6b,NaN,concept status,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4853,4853,https://data.cultureelerfgoed.nl/term/id/abr/0ca36c41-de5b-4ba1-90b3-c0130c81787a,NaN,NaN,stuifzandvlakte,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,VSZ,NaN,NaN,NaN,NaN
4854,4854,https://data.cultureelerfgoed.nl/term/id/abr/fe9a85f8-d7b2-4314-86b6-e8ed7866624f,NaN,NaN,heuvelrug door tectoniek ontstaan,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,RHE,NaN,NaN,NaN,NaN
4855,4855,https://data.cultureelerfgoed.nl/term/id/abr/6acfbc78-488f-4b05-b86e-a2c9903b95be,NaN,NaN,veenvlakte (ontgonnen),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,VOV,NaN,NaN,NaN,NaN
4856,4856,https://data.cultureelerfgoed.nl/term/id/abr/c4dff91e-2427-48d7-829a-2816711bdd23,NaN,NaN,oeverwal,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,ROE,NaN,NaN,NaN,NaN


In [49]:
def getCode(row):
    lst_codes = ['https://data.cultureelerfgoed.nl/vocab/id/abr#hasSpecificAbrCode', 
                 'https://data.cultureelerfgoed.nl/vocab/id/abr#hasSpecificAbrCode@nl', 
                 'https://data.cultureelerfgoed.nl/vocab/id/rce#hasCode', 
                 'https://data.cultureelerfgoed.nl/vocab/id/rce#hasCode@nl']
    for n in range(0, len(lst_codes)-1):
        if pd.notnull(row[lst_codes[n]]):
            return (row[lst_codes[n]])
    return None


lst = []
lst_cnc = []
lst_cd = []
lst_id = []

for index, row in df_abr.iterrows():
    if pd.notnull(row.scheme):
        lst.clear()
        lst.append(row.uri) 
        lst_id.clear()
        lst_id.append(row.ID) 
        lst_cnc.clear()
        lst_cnc.append(row.scheme)
        lst_cd.clear()
        lst_cd.append(getCode(row)) 
    elif pd.notnull(row.concept):
        lst = lst[:1]
        lst.append(row.uri) 
        lst_id = lst_id[:1]
        lst_id.append(row.ID) 
        lst_cnc = lst_cnc[:1]
        lst_cnc.append(row.concept)
        lst_cd = lst_cd[:1]
        lst_cd.append(getCode(row)) 
    else:
        rng = range(1,7)
        for n in rng:
            if pd.notnull(row['concept.' + str(n)]):
                lst = lst[:n+1]
                lst.append(row.uri) 
                lst_id = lst_id[:n+1]
                lst_id.append(row.ID) 
                lst_cnc = lst_cnc[:n+1]
                lst_cnc.append(row['concept.' + str(n)])
                lst_cd = lst_cd[:n+1]
                lst_cd.append(getCode(row))
    
    #print(lst)
    df_abr.at[index, 'uris'] = lst
    df_abr.at[index, 'parent_uri'] = lst[len(lst)-2] if len(lst)-2 > -1 else None
    df_abr.at[index, 'concept'] = lst_cnc[len(lst_cnc)-1] if len(lst_cnc)-1 > -1 else None
    df_abr.at[index, 'parent_Concept'] = lst_cnc[len(lst_cnc)-2] if len(lst_cnc)-2 > -1 else None
    df_abr.at[index, 'code'] = lst_cd[len(lst_cd)-1] if len(lst_cd)-1 > -1 else None
    df_abr.at[index, 'parent_code'] = lst_cd[len(lst_cd)-2] if len(lst_cd)-2 > -1 else None
    df_abr.at[index, 'parentID'] = lst_id[len(lst_id)-2] if len(lst_id)-2 > -1 else None
        
    
parent = 'https://data.cultureelerfgoed.nl/term/id/abr/2dcb88e1-e777-4cff-babc-d97485b42b10'
df_abr[df_abr.uris.apply(lambda x: parent in x)].head()

,ID,uri,scheme,concept,concept.1,concept.2,concept.3,concept.4,concept.5,concept.6,...,https://data.cultureelerfgoed.nl/vocab/id/rce#hasSourceId,https://data.cultureelerfgoed.nl/vocab/id/rce#hasSourceId@nl,https://data.cultureelerfgoed.nl/vocab/id/rce#hasSpecification,https://data.cultureelerfgoed.nl/vocab/id/rce#hasSpecification@nl,uris,parent_uri,parent_Concept,code,parent_code,parentID
31,31,https://data.cultureelerfgoed.nl/term/id/abr/2dcb88e1-e777-4cff-babc-d97485b42b10,NaN,materialen,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"[https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed, https://data.cultureelerfgoed.nl/term/id/abr/2dcb88e1-e777-4cff-babc-d97485b42b10]",https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed,Archeologisch Basis Register,NaN,NaN,30.0
32,32,https://data.cultureelerfgoed.nl/term/id/abr/vervallen_materialen,NaN,vervallen materialen,vervallen materialen,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"[https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed, https://data.cultureelerfgoed.nl/term/id/abr/2dcb88e1-e777-4cff-babc-d97485b42b10, https://data.cultureelerfgoed.nl/term/id/abr/vervallen_materialen]",https://data.cultureelerfgoed.nl/term/id/abr/2dcb88e1-e777-4cff-babc-d97485b42b10,materialen,NaN,NaN,31.0
33,33,https://data.cultureelerfgoed.nl/term/id/abr/3c6900e6-a665-4603-b123-4dfb84afdc6e,NaN,magering,NaN,magering,NaN,NaN,NaN,NaN,...,NaN,NaN,was ABR specifiek niet in KNA.,NaN,"[https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed, https://data.cultureelerfgoed.nl/term/id/abr/2dcb88e1-e777-4cff-babc-d97485b42b10, https://data.cultureelerfgoed.nl/term/id/abr/vervallen_materialen, https://data.cultureelerfgoed.nl/term/id/abr/3c6900e6-a665-4603-b123-4dfb84afdc6e]",https://data.cultureelerfgoed.nl/term/id/abr/vervallen_materialen,vervallen materialen,KMA,NaN,32.0
34,34,https://data.cultureelerfgoed.nl/term/id/abr/134b1acb-18ec-455a-bc0b-f237cfb3b2f9,NaN,calciet,NaN,NaN,calciet,NaN,NaN,NaN,...,NaN,NaN,vervallen,NaN,"[https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed, https://data.cultureelerfgoed.nl/term/id/abr/2dcb88e1-e777-4cff-babc-d97485b42b10, https://data.cultureelerfgoed.nl/term/id/abr/vervallen_materialen, https://data.cultureelerfgoed.nl/term/id/abr/3c6900e6-a665-4603-b123-4dfb84afdc6e, https://data.cultureelerfgoed.nl/term/id/abr/134b1acb-18ec-455a-bc0b-f237cfb3b2f9]",https://data.cultureelerfgoed.nl/term/id/abr/3c6900e6-a665-4603-b123-4dfb84afdc6e,magering,KMC,KMA,33.0
35,35,https://data.cultureelerfgoed.nl/term/id/abr/bfd2a9c3-9f59-472a-ad5b-7ba32846c565,NaN,organische magering,NaN,NaN,organische magering,NaN,NaN,NaN,...,NaN,NaN,vervallen,NaN,"[https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed, https://data.cultureelerfgoed.nl/term/id/abr/2dcb88e1-e777-4cff-babc-d97485b42b10, https://data.cultureelerfgoed.nl/term/id/abr/vervallen_materialen, https://data.cultureelerfgoed.nl/term/id/abr/3c6900e6-a665-4603-b123-4dfb84afdc6e, https://data.cultureelerfgoed.nl/term/id/abr/bfd2a9c3-9f59-472a-ad5b-7ba32846c565]",https://data.cultureelerfgoed.nl/term/id/abr/3c6900e6-a665-4603-b123-4dfb84afdc6e,magering,KMO,KMA,33.0


In [50]:
df_abr.rename(columns={"ID": "primary_key", "altLabel@nl": "label", "scopeNote@nl": 'note', "https://data.cultureelerfgoed.nl/vocab/id/abr#hasBeginYear":"beginjaar", "https://data.cultureelerfgoed.nl/vocab/id/abr#hasEndYear": "eindjaar", 
                       "https://data.cultureelerfgoed.nl/vocab/id/rce#hasSourceId": "bron", "https://data.cultureelerfgoed.nl/vocab/id/rce#hasSpecification": "opmerkingen"}, inplace=True)
df_abr.columns

Index(['primary_key', 'uri', 'scheme', 'concept', 'concept.1', 'concept.2',
       'concept.3', 'concept.4', 'concept.5', 'concept.6', 'concept.7',
       'label', 'hiddenLabel@nl', 'type', 'note', 'beginjaar', 'eindjaar',
       'https://data.cultureelerfgoed.nl/vocab/id/abr#hasSpecificAbrCode',
       'https://data.cultureelerfgoed.nl/vocab/id/abr#hasSpecificAbrCode@nl',
       'https://data.cultureelerfgoed.nl/vocab/id/rce#hasCode',
       'https://data.cultureelerfgoed.nl/vocab/id/rce#hasCode@nl', 'bron',
       'https://data.cultureelerfgoed.nl/vocab/id/rce#hasSourceId@nl',
       'opmerkingen',
       'https://data.cultureelerfgoed.nl/vocab/id/rce#hasSpecification@nl',
       'uris', 'parent_uri', 'parent_Concept', 'code', 'parent_code',
       'parentID'],
      dtype='object')

In [51]:
df_abr = df_abr[['primary_key', 'concept', 'code', 'label', 'note', 'beginjaar', 'eindjaar', 'bron', 'opmerkingen', 'parentID', 'uri', 'uris']]
df_abr

,primary_key,concept,code,label,note,beginjaar,eindjaar,bron,opmerkingen,parentID,uri,uris
0,0,Algemene Lijsten,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://data.cultureelerfgoed.nl/term/id/abr/AlgemeneLijsten,https://data.cultureelerfgoed.nl/term/id/abr/AlgemeneLijsten
1,1,bronhouder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,https://data.cultureelerfgoed.nl/term/id/abr/bc241422-ee0d-4c44-aead-94a25b5e9fa4,"[https://data.cultureelerfgoed.nl/term/id/abr/AlgemeneLijsten, https://data.cultureelerfgoed.nl/term/id/abr/bc241422-ee0d-4c44-aead-94a25b5e9fa4]"
2,2,Stichting Infrastructuur Kwaliteitsborging Bodembeheer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,https://data.cultureelerfgoed.nl/term/id/abr/17e56d62-b1f2-42e1-afbf-578d7b018380,"[https://data.cultureelerfgoed.nl/term/id/abr/AlgemeneLijsten, https://data.cultureelerfgoed.nl/term/id/abr/bc241422-ee0d-4c44-aead-94a25b5e9fa4, https://data.cultureelerfgoed.nl/term/id/abr/17e56d62-b1f2-42e1-afbf-578d7b018380]"
3,3,Rijksdienst voor het Cultureel Erfgoed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,https://data.cultureelerfgoed.nl/term/id/abr/1576adc6-1af1-44d0-9d46-593eb6bcad09,"[https://data.cultureelerfgoed.nl/term/id/abr/AlgemeneLijsten, https://data.cultureelerfgoed.nl/term/id/abr/bc241422-ee0d-4c44-aead-94a25b5e9fa4, https://data.cultureelerfgoed.nl/term/id/abr/1576adc6-1af1-44d0-9d46-593eb6bcad09]"
4,4,concept status,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,https://data.cultureelerfgoed.nl/term/id/abr/d159858d-d746-4de5-98be-82cc5b697f6b,"[https://data.cultureelerfgoed.nl/term/id/abr/AlgemeneLijsten, https://data.cultureelerfgoed.nl/term/id/abr/d159858d-d746-4de5-98be-82cc5b697f6b]"
...,...,...,...,...,...,...,...,...,...,...,...,...
4853,4853,stuifzandvlakte,None,NaN,NaN,NaN,NaN,NaN,NaN,4801.0,https://data.cultureelerfgoed.nl/term/id/abr/0ca36c41-de5b-4ba1-90b3-c0130c81787a,"[https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed, https://data.cultureelerfgoed.nl/term/id/abr/82638dc4-8095-4c54-a7ff-49ac32ad5361, https://data.cultureelerfgoed.nl/term/id/abr/0ca36c41-de5b-4ba1-90b3-c0130c81787a]"
4854,4854,heuvelrug door tectoniek ontstaan,None,NaN,NaN,NaN,NaN,NaN,NaN,4801.0,https://data.cultureelerfgoed.nl/term/id/abr/fe9a85f8-d7b2-4314-86b6-e8ed7866624f,"[https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed, https://data.cultureelerfgoed.nl/term/id/abr/82638dc4-8095-4c54-a7ff-49ac32ad5361, https://data.cultureelerfgoed.nl/term/id/abr/fe9a85f8-d7b2-4314-86b6-e8ed7866624f]"
4855,4855,veenvlakte (ontgonnen),None,NaN,NaN,NaN,NaN,NaN,NaN,4801.0,https://data.cultureelerfgoed.nl/term/id/abr/6acfbc78-488f-4b05-b86e-a2c9903b95be,"[https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed, https://data.cultureelerfgoed.nl/term/id/abr/82638dc4-8095-4c54-a7ff-49ac32ad5361, https://data.cultureelerfgoed.nl/term/id/abr/6acfbc78-488f-4b05-b86e-a2c9903b95be]"
4856,4856,oeverwal,None,NaN,NaN,NaN,NaN,NaN,NaN,4801.0,https://data.cultureelerfgoed.nl/term/id/abr/c4dff91e-2427-48d7-829a-2816711bdd23,"[https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed, https://data.cultureelerfgoed.nl/term/id/abr/82638dc4-8095-4c54-a7ff-49ac32ad5361, https://data.cultureelerfgoed.nl/term/id/abr/c4dff91e-2427-48d7-829a-2816711bdd23]"


In [52]:
df_abr.to_excel(outputExcel)

In [59]:
engine = create_engine(config.SQLALCHEMY_DATABASE_URI)
with engine.connect() as connection:
    df = pd.read_sql_query('SELECT * from "Def_ABR"', connection)

df

,herkomst,key,soort,brondata,_id,primary_key,concept,code,label,note,beginjaar,eindjaar,bron,opmerkingen,uri,parentID,uris
0,None,None,None,None,None,0,Algemene Lijsten,None,None,None,NaN,NaN,None,None,https://data.cultureelerfgoed.nl/term/id/abr/AlgemeneLijsten,NaN,https://data.cultureelerfgoed.nl/term/id/abr/AlgemeneLijsten
1,None,None,None,None,None,1,bronhouder,None,None,None,NaN,NaN,None,None,https://data.cultureelerfgoed.nl/term/id/abr/bc241422-ee0d-4c44-aead-94a25b5e9fa4,0.0,"{https://data.cultureelerfgoed.nl/term/id/abr/AlgemeneLijsten,https://data.cultureelerfgoed.nl/term/id/abr/bc241422-ee0d-4c44-aead-94a25b5e9fa4}"
2,None,None,None,None,None,2,Stichting Infrastructuur Kwaliteitsborging Bodembeheer,None,None,None,NaN,NaN,None,None,https://data.cultureelerfgoed.nl/term/id/abr/17e56d62-b1f2-42e1-afbf-578d7b018380,1.0,"{https://data.cultureelerfgoed.nl/term/id/abr/AlgemeneLijsten,https://data.cultureelerfgoed.nl/term/id/abr/bc241422-ee0d-4c44-aead-94a25b5e9fa4,https://data.cultureelerfgoed.nl/term/id/abr/17e56d62-b1f2-42e1-afbf-578d7b018380}"
3,None,None,None,None,None,3,Rijksdienst voor het Cultureel Erfgoed,None,None,None,NaN,NaN,None,None,https://data.cultureelerfgoed.nl/term/id/abr/1576adc6-1af1-44d0-9d46-593eb6bcad09,1.0,"{https://data.cultureelerfgoed.nl/term/id/abr/AlgemeneLijsten,https://data.cultureelerfgoed.nl/term/id/abr/bc241422-ee0d-4c44-aead-94a25b5e9fa4,https://data.cultureelerfgoed.nl/term/id/abr/1576adc6-1af1-44d0-9d46-593eb6bcad09}"
4,None,None,None,None,None,4,concept status,None,None,None,NaN,NaN,None,None,https://data.cultureelerfgoed.nl/term/id/abr/d159858d-d746-4de5-98be-82cc5b697f6b,0.0,"{https://data.cultureelerfgoed.nl/term/id/abr/AlgemeneLijsten,https://data.cultureelerfgoed.nl/term/id/abr/d159858d-d746-4de5-98be-82cc5b697f6b}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4853,None,None,None,None,None,4853,stuifzandvlakte,None,None,None,NaN,NaN,None,None,https://data.cultureelerfgoed.nl/term/id/abr/0ca36c41-de5b-4ba1-90b3-c0130c81787a,4801.0,"{https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed,https://data.cultureelerfgoed.nl/term/id/abr/82638dc4-8095-4c54-a7ff-49ac32ad5361,https://data.cultureelerfgoed.nl/term/id/abr/0ca36c41-de5b-4ba1-90b3-c0130c81787a}"
4854,None,None,None,None,None,4854,heuvelrug door tectoniek ontstaan,None,None,None,NaN,NaN,None,None,https://data.cultureelerfgoed.nl/term/id/abr/fe9a85f8-d7b2-4314-86b6-e8ed7866624f,4801.0,"{https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed,https://data.cultureelerfgoed.nl/term/id/abr/82638dc4-8095-4c54-a7ff-49ac32ad5361,https://data.cultureelerfgoed.nl/term/id/abr/fe9a85f8-d7b2-4314-86b6-e8ed7866624f}"
4855,None,None,None,None,None,4855,veenvlakte (ontgonnen),None,None,None,NaN,NaN,None,None,https://data.cultureelerfgoed.nl/term/id/abr/6acfbc78-488f-4b05-b86e-a2c9903b95be,4801.0,"{https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed,https://data.cultureelerfgoed.nl/term/id/abr/82638dc4-8095-4c54-a7ff-49ac32ad5361,https://data.cultureelerfgoed.nl/term/id/abr/6acfbc78-488f-4b05-b86e-a2c9903b95be}"
4856,None,None,None,None,None,4856,oeverwal,None,None,None,NaN,NaN,None,None,https://data.cultureelerfgoed.nl/term/id/abr/c4dff91e-2427-48d7-829a-2816711bdd23,4801.0,"{https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed,https://data.cultureelerfgoed.nl/term/id/abr/82638dc4-8095-4c54-a7ff-49ac32ad5361,https://data.cultureelerfgoed.nl/term/id/abr/c4dff91e-2427-48d7-829a-2816711bdd23}"


In [58]:
engine = create_engine(config.SQLALCHEMY_DATABASE_URI)
with engine.connect() as connection:
    connection.execute('TRUNCATE TABLE "Def_ABR"')
    df_abr.to_sql('Def_ABR', con=connection, if_exists='append', index=False)

In [63]:
engine = create_engine(config.SQLALCHEMY_DATABASE_URI)
with engine.connect() as connection:
    df = pd.read_sql_query('SELECT * from "Def_ABR" WHERE "parentID" = 31', connection)

df

,herkomst,key,soort,brondata,_id,primary_key,concept,code,label,note,beginjaar,eindjaar,bron,opmerkingen,uri,parentID,uris
0,None,None,None,None,None,32,vervallen materialen,None,None,None,None,None,None,None,https://data.cultureelerfgoed.nl/term/id/abr/vervallen_materialen,31,"{https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed,https://data.cultureelerfgoed.nl/term/id/abr/2dcb88e1-e777-4cff-babc-d97485b42b10,https://data.cultureelerfgoed.nl/term/id/abr/vervallen_materialen}"
1,None,None,None,None,None,41,metaal,MXX,None,"De chemische samenstelling van metalen voorwerpen kan zeer uiteenlopen. In de eerste plaats is deze afhankelijk van de legering, in de tweede plaats van de productiemethode en in de derde plaats (op het niveau van sporenelementen of isotopen) is deze afhankelijk van verschillen in herkomst van grondstoffen en brandstof. In Nederland is het gebruik van metaal in de prehistorie vrij zeldzaam; go...",None,None,None,None,https://data.cultureelerfgoed.nl/term/id/abr/bf251e68-ef3c-46e7-840d-3762166b9a8d,31,"{https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed,https://data.cultureelerfgoed.nl/term/id/abr/2dcb88e1-e777-4cff-babc-d97485b42b10,https://data.cultureelerfgoed.nl/term/id/abr/bf251e68-ef3c-46e7-840d-3762166b9a8d}"
2,None,None,None,None,None,55,organisch,OXX,None,"Hout, plantenresten en textielvezels van plantaardige oorsprong zijn opgebouwd uit verschillende soorten organische verbindingen, met als hoofdbestanddelen koolstof, zuurstof, waterstof en stikstof. De exacte samenstelling en de opbouw en onderlinge samenhang van de celstructuur verschilt van soort tot soort en van onderdeel tot onderdeel en kan door degradatie of verhitting worden beïnvloed. ...",None,None,None,None,https://data.cultureelerfgoed.nl/term/id/abr/e49a304e-08a3-4751-94ee-1bb507e49c72,31,"{https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed,https://data.cultureelerfgoed.nl/term/id/abr/2dcb88e1-e777-4cff-babc-d97485b42b10,https://data.cultureelerfgoed.nl/term/id/abr/e49a304e-08a3-4751-94ee-1bb507e49c72}"
3,None,None,None,None,None,89,monsters,MA,None,Grondmonsters.,None,None,None,None,https://data.cultureelerfgoed.nl/term/id/abr/a443bc52-d653-4eca-82cb-4351361e2dfb,31,"{https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed,https://data.cultureelerfgoed.nl/term/id/abr/2dcb88e1-e777-4cff-babc-d97485b42b10,https://data.cultureelerfgoed.nl/term/id/abr/a443bc52-d653-4eca-82cb-4351361e2dfb}"
4,None,None,None,None,None,90,glas,GLS,None,"Glas bestaat uit een mengsel van silica met calcium en natrium of kalium, dat zodanig is verhit dat een onregelmatige netwerkstructuur is ontstaan. Soms zijn er stoffen toegevoegd of speciale smelttechnieken gebruikt om het glas een specifieke kleur te geven. De gebruikte grondstoffen en de eigenschappen van glas verschillen sterk per periode.",None,None,None,None,https://data.cultureelerfgoed.nl/term/id/abr/df8bf8b3-b4a9-4e10-b5f9-59234cc42bb0,31,"{https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed,https://data.cultureelerfgoed.nl/term/id/abr/2dcb88e1-e777-4cff-babc-d97485b42b10,https://data.cultureelerfgoed.nl/term/id/abr/df8bf8b3-b4a9-4e10-b5f9-59234cc42bb0}"
5,None,None,None,None,None,91,keramiek,KER,None,"Keramiek bestaat uit klei en andere mineralen die zodanig zijn verhit dat er chemisch/mineralogische veranderingen zijn opgetreden die ervoor hebben gezorgd dat het materiaal stevigheid heeft en (in sommige gevallen) waterdicht is. De chemisch/mineralogische samenstelling van keramiek is afhankelijk van de gebruikte grondstoffen, baktemperatuur en ovenconfiguratie (oxiderend vs. reducerend mil...",None,None,None,None,https://data.cultureelerfgoed.nl/term/id/abr/6d7809db-db25-4c09-90c3-e455f61148a9,31,"{https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed,https://data.cultureelerfgoed.nl/term/id/abr/2dcb88e1-e777-4cff-babc-d97485b

In [89]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Text, ForeignKey
from sqlalchemy.orm import relationship, Session

Base = declarative_base()


class WasstraatModel(Base):
    __abstract__ = True

    herkomst = Column(Text)
    key = Column(Text)
    soort = Column(String(80))
    brondata = Column(Text)
    uuid = Column('_id', String)

class ABR(Base):
    __tablename__ = 'Def_ABR'
    primary_key = Column(Integer, primary_key=True, autoincrement=True)
    concept = Column(String(256))
    code = Column(String(80))
    label = Column(String(256))
    note = Column(Text)
    beginjaar = Column(Integer)
    eindjaar = Column(Integer)
    bron = Column(String(256))
    opmerkingen = Column(Text)
    uri = Column(String(256))
    uris = Column(Text)
    parentID = Column(ForeignKey('Def_ABR.primary_key', deferrable=True), index=True)
    parent = relationship('ABR')

    def __repr__(self):
        return f"({self.code}) " if self.code else "" + self.concept


engine = create_engine(config.SQLALCHEMY_DATABASE_URI)    
sess = Session(engine)

topq = sess.query(ABR)
topq = topq.filter(ABR.primary_key == 956)
topq = topq.cte('cte', recursive=True)

bottomq = sess.query(ABR)
bottomq = bottomq.join(topq, ABR.parentID == topq.c.primary_key)

recursive_q = topq.union(bottomq)
q = sess.query(recursive_q)

for row in q:
    print("ID: ", row["primary_key"], "parentID: ", row["parentID"], "concept:", row["concept"], )


ID:  956 parentID:  30 concept: periodes
ID:  957 parentID:  956 concept: Prehistorie
ID:  990 parentID:  956 concept: onbekend
ID:  991 parentID:  956 concept: Protohistorie
ID:  1006 parentID:  956 concept: Historie
ID:  958 parentID:  957 concept: IJzertijd
ID:  962 parentID:  957 concept: Metaaltijden
ID:  963 parentID:  957 concept: Steentijd
ID:  984 parentID:  957 concept: Bronstijd
ID:  992 parentID:  991 concept: Romeinse Tijd
ID:  1007 parentID:  1006 concept: Recent
ID:  1008 parentID:  1006 concept: Middeleeuwen
ID:  1021 parentID:  1006 concept: Nieuwe Tijd
ID:  959 parentID:  958 concept: Midden IJzertijd
ID:  960 parentID:  958 concept: Vroege IJzertijd
ID:  961 parentID:  958 concept: Late IJzertijd
ID:  964 parentID:  963 concept: Mesolithicum
ID:  968 parentID:  963 concept: Paleolithicum
ID:  974 parentID:  963 concept: Neolithicum
ID:  985 parentID:  984 concept: Late Bronstijd
ID:  986 parentID:  984 concept: Vroege Bronstijd
ID:  987 parentID:  984 concept: Midden